In [1]:
import torch
IsTPU = False
batch_size = 4
device = 'cuda'
dtype = torch.float32
speech_path = 'data/train_mp3s/'
data_path = 'data/train.csv'
num_workers = 16
clip = 4e-5
lr = 4e-7
output_path = ''
model_path = '/home/zhenlan/Desktop/Projects/Bengali ASR/wav2vec_CTC/wav2vec.pth'
background_audio = "/home/zhenlan/Desktop/Projects/Bengali ASR/ESC-50-master/audio"
background_music = "/home/zhenlan/Desktop/Projects/Bengali ASR/ESC-50-master/music"
add_data_path = 'dlsprint/train.csv'
add_audio = 'dlsprint/train_files/'
add_data_path2 = 'text_train'
add_audio2 = 'RESPIN/'

In [2]:
#from whisper_jax import FlaxWhisperForConditionalGeneration
from functions import *
import torch.nn as nn
from functools import partial
from transformers.modeling_outputs import BaseModelOutput
from transformers import WhisperForConditionalGeneration
from transformers.generation.configuration_utils import GenerationConfig
from transformers import AutoTokenizer,Wav2Vec2Processor
from torch.optim import AdamW,Adam,SGD
from torch.nn.utils import clip_grad_value_
from torch.cuda.amp import GradScaler
import time
import warnings
warnings.filterwarnings('ignore')

In [3]:
model = torch.load(model_path).to(device)

In [4]:
paras = model.parameters()
opt = AdamW(paras,lr = lr,amsgrad=True)
#opt = Adam(paras,lr = lr)
#opt = SGD(paras,lr = lr)

In [5]:
epochs = 1
verbose = 10
processor = Wav2Vec2Processor.from_pretrained("arijitx/wav2vec2-xls-r-300m-bengali")
tokenizer = processor.tokenizer
feature_extractor = processor.feature_extractor
text = pd.read_csv(data_path)
add_data = pd.read_csv(add_data_path)
# data source 2
df = pd.read_csv(add_data_path2,names=['code'])
df[['code','sentence']] = df["code"].str.split(" ", n=1, expand=True)
df.sentence = df.sentence.str.strip()

In [6]:
# "https://github.com/karoldvl/ESC-50/archive/master.zip"
from audiomentations import (
    AddBackgroundNoise,
    AddGaussianNoise,
    Compose,
    Gain,
    GainTransition,
    OneOf,
    PitchShift,
    PolarityInversion,
    TimeStretch,
    OneOf,
    )

# define augmentation
augmentation = Compose(
    [
        TimeStretch(min_rate=0.9, max_rate=1.1, p=1, leave_length_unchanged=False),
        Gain(min_gain_db=-6, max_gain_db=6, p=1),
        GainTransition(min_gain_db=-6, max_gain_db=6),
        PitchShift(min_semitones=-4, max_semitones=4, p=1),
        OneOf([AddBackgroundNoise(sounds_path=background_audio, min_snr_in_db=1.0, max_snr_in_db=5.0, noise_transform=PolarityInversion(), p=1.0),\
              AddBackgroundNoise(sounds_path=background_music, min_snr_in_db=1.0, max_snr_in_db=5.0, noise_transform=PolarityInversion(), p=1.0)]),
        AddGaussianNoise(min_amplitude=0.005, max_amplitude=0.015, p=1),
    ]
)


In [7]:
dataset1 = AudioDataset(text,speech_path,lambda x:x.id+'.mp3',\
                       augmentation,orig_sr=32000, target_sr=16000)
dataset2 = AudioDataset(add_data,add_audio,\
                        lambda x:x.path,augmentation,orig_sr=48000, target_sr=16000)
dataset3 = AudioDataset(df,add_audio2,\
                        lambda x:x.code.split('_')[-1] + '.wav',augmentation,orig_sr=16000, target_sr=16000)
# dataset1 = AudioDataset(text,speech_path,lambda x:x.id+'.mp3',\
#                        orig_sr=32000, target_sr=16000)
# dataset2 = AudioDataset(add_data,add_audio,\
#                         lambda x:x.path,orig_sr=48000, target_sr=16000)
# dataset3 = AudioDataset(df,add_audio2,\
#                         lambda x:x.code.split('_')[-1] + '.wav',orig_sr=16000, target_sr=16000)                        
dataset = Add2Data(dataset1,dataset2)
dataset = Add2Data(dataset,dataset3)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, \
                            collate_fn=partial(collate_fn_pt_wav2vec,tokenizer=tokenizer,feature_extractor=feature_extractor))

In [8]:
#%debug
# audio,input_ids,attention_mask = next(iter(train_loader))
# audio,input_ids,attention_mask = torch.asarray(audio,dtype=torch.float32,device=device),torch.asarray(input_ids,dtype=torch.long,device=device),torch.asarray(attention_mask,dtype=torch.float32,device=device)
# audio.shape,input_ids.shape
# audio,input_ids,attention_mask = jnp.array(audio,dtype=dtype),jnp.array(input_ids),jnp.array(attention_mask)

In [9]:
use_amp = True
model.train()
np.random.seed()
train_loss = 0
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
# Training #
for j,(audio,input_ids,attention_mask) in enumerate(train_loader):
    audio,input_ids,attention_mask = torch.asarray(audio,device=device),\
                                     torch.asarray(input_ids,device=device),\
                                     torch.asarray(attention_mask,device=device)
    with torch.autocast(device_type=device, dtype=torch.float16, enabled=use_amp):
        loss = model(audio,attention_mask=attention_mask,labels=input_ids).loss
    train_loss += loss.item()
    scaler.scale(loss).backward()
    scaler.unscale_(opt)
    clip_grad_value_(paras,clip)
    scaler.step(opt)
    scaler.update()
    opt.zero_grad()
    #print(torch.any(torch.isnan(logits)),scaler.get_scale())
    if j>0 and j%verbose == 0:
        train_loss /= verbose
        print(f"iterations:{j}, loss: {train_loss:.3f}")
        train_loss = 0
        skip = 0
    if j>2000:
        break

iterations:10, loss: inf
iterations:20, loss: 3.464
iterations:30, loss: 3.701
iterations:40, loss: 3.526
iterations:50, loss: inf
iterations:60, loss: 3.500
iterations:70, loss: inf
iterations:80, loss: inf
iterations:90, loss: inf
iterations:100, loss: inf
iterations:110, loss: inf
iterations:120, loss: inf
iterations:130, loss: 3.478


KeyboardInterrupt: 

In [ ]:
torch.save(model, model_path)